<h2>Indexing the document collection</h2>

<h3>Settings & utilities</h3>

In [1]:
#imports
import xml.etree.ElementTree as ET
import os
import string
import numpy as np
import spacy #lemmatization
import pickle #serialize object read/write files
import time

<h3>Load document collection</h3>

In [2]:
#path to the document collection
collection_path = 'xmldocs/'

def getDocumentList(path):
    return os.listdir(path)
    
#generate document name list
doc_list = getDocumentList(collection_path)

<h2>Processing text</h2>

<h3>Load stopwords</h3>

In [3]:
#Load a stopword list NLTK
gist_file = open("stopwords.txt", "r")
try:
    content = gist_file.read()
    stopwords = content.split(",")
finally:
    gist_file.close()

<h3>Text Processing Function</h3>

In [4]:
#Process text to meet the IR requirements
use_lemmatization = True

# Lemmatization
# https://www.analyticsvidhya.com/blog/2019/08
#/how-to-remove-stopwords-text-normalization-nltk-spacy-gensim-python
#/?utm_source=blog&utm_medium=information-retrieval-using-word2vec-based-vector-space-model
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000

def lemmatize(x):
    return ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)])
    
    
#Returns a list of relevant terms
def processText(text):
    #remove punctuation and stopwords
    #remove single punctuation characters, remove points (not separated from string), lower case all 
    if not isinstance(text, str) :
        return []
    #remove punctuation
    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).strip()
    #remove unnecessary whitespace
    text = text.replace("  ", " ")
    #lower the text
    text = text.lower()
    #lemmatize
    if use_lemmatization:
        text = lemmatize(text)
    return list(filter(lambda el: el not in stopwords, text.split()))

<h2>Class for XML documents</h2>

In [5]:
class XmlFile:
    def __init__(self, xml, xml_id, field_dict, processText):
        self.field_dict = field_dict
        tree = ET.parse(xml)
        root = tree.getroot()
        self.no_process_field_exception = ['img_dir']
        for id in root.iter(xml_id):
            self.id = id.text if id.text is not None else ''
        for xml_name, field_name in field_dict.items():
            for content in root.iter(xml_name):
                if field_name not in self.no_process_field_exception:
                    self.__dict__[field_name] = processText(content.text) if content.text is not None else []
                else:
                    self.__dict__[field_name] = content.text if content.text is not None else ''
                
    def __getTF__(self, processed_text):
        #f/max_occurance of most frequent term
        num_terms = len(processed_text)
        tf_index = {}
        max_term_count = 0
        for term in processed_text:
            if term not in tf_index: #save some time
                term_count = processed_text.count(term)
                if term_count > max_term_count:
                    max_term_count = term_count
                tf_index[term] = term_count/num_terms
        #Normalization does not affect the results in this collection
        #for term in tf_index.keys():
            #tf_index[term] = tf_index[term] / max_term_count
        return tf_index

In [6]:
#relevant document tags 
xml_doc_id = 'DOCID'
xml_doc_fields = {'HEADLINE' : 'processed_head', 
                'TEXT' : 'processed_body',
                'IMGDIR' : 'img_dir'  
                 }

class XmlDoc(XmlFile):
    def __init__(self, xml):
        super().__init__(xml, xml_doc_id, xml_doc_fields, processText)       
        self.tf_head = self.__getTF__(self.processed_head)
        self.tf_body = self.__getTF__(self.processed_body)
        self.tf_text = self.__getTF__(self.processed_head + self.processed_body)

<h2>Document Collection</h2>

In [7]:
class DocumentCollection:
    def __init__(self, path, doc_list, 
                 model_structure_file_path = 'retrieval_model_structures/'
                ):
        self.docs_filename = model_structure_file_path+'document_collection.docs'
        self.index_filename = model_structure_file_path+'document_collection.inverted_index'
        self.global_ft_filename = model_structure_file_path+'document_collection.global_tf'
        self.total_terms_in_collection_filename = model_structure_file_path+'document_collection.total_terms_in_collection'
        self.idf_filename = model_structure_file_path+'document_collection.idf'
        self.avgdl_filename = model_structure_file_path+'document_collection.avgdl'
        self.path = path
        self.division_factor = 10
        #Inverted index 
            
    def loadFromFile(self):
        #Docs
        self.docs = {}
        for i in range(self.division_factor):
            with open(self.docs_filename+'.part_'+str(i), 'rb') as dictionary_file:
                self.docs.update(pickle.load(dictionary_file))
        #Inverted Index
        with open(self.index_filename, 'rb') as dictionary_file:
            self.inverted_index = pickle.load(dictionary_file)
        #Global TF
        with open(self.global_ft_filename, 'rb') as dictionary_file:
            self.global_tf = pickle.load(dictionary_file)
        #Idf
        with open(self.idf_filename, 'rb') as dictionary_file:
            self.idf = pickle.load(dictionary_file)
        #total terms in collection
        with open(self.total_terms_in_collection_filename, 'rb') as dictionary_file:
            self.total_terms_in_collection = pickle.load(dictionary_file)
        #Avg Doc Length
        with open(self.avgdl_filename, 'rb') as dictionary_file:
            self.avgdl = pickle.load(dictionary_file)
    
    def computeAndDump(self, doc_list):
        division_lenght = round(len(doc_list)/self.division_factor)
        #Docs
        self.docs = {}
        for i in range(self.division_factor):
            docs = {}
            st = i*division_lenght
            en = (i+1)*division_lenght
            for d in doc_list[st:en]:
                xml_document = XmlDoc(self.path + '/' + d)
                docs[xml_document.id] = xml_document
            with open(self.docs_filename+'.part_'+str(i), 'wb') as dictionary_file:
                pickle.dump(docs, dictionary_file)
            self.docs.update(docs)
        
        self.global_tf = {}
        self.total_terms_in_collection = 0
        self.inverted_index = {}
        for id in self.docs.keys():
            for term in self.docs[id].processed_head + self.docs[id].processed_body:
                self.total_terms_in_collection += 1
                if term not in self.global_tf:
                    self.global_tf[term] = 1
                else: self.global_tf[term] += 1
                if term in self.inverted_index:
                    self.inverted_index[term].add(id)
                else: self.inverted_index[term] = {id}
        for term in self.global_tf.keys():
            self.global_tf[term] = self.global_tf[term] / self.total_terms_in_collection
        
        #Table of IDF
        self.idf = {}
        for term in self.inverted_index.keys():
            self.idf[term] = np.log(len(self.docs) / len(self.inverted_index[term]))
            
        #Average document length
        self.avgdl = np.mean([len(item.processed_head + item.processed_body) for k, item in self.docs.items()])
        
        with open(self.index_filename, 'wb') as dictionary_file:
            pickle.dump(self.inverted_index, dictionary_file)
            
        with open(self.global_ft_filename, 'wb') as dictionary_file:
            pickle.dump(self.global_tf, dictionary_file)
            
        with open(self.idf_filename, 'wb') as dictionary_file:
            pickle.dump(self.idf, dictionary_file)
            
        with open(self.total_terms_in_collection_filename, 'wb') as dictionary_file:
            pickle.dump(self.total_terms_in_collection, dictionary_file)
            
        with open(self.avgdl_filename, 'wb') as dictionary_file:
            pickle.dump(self.avgdl, dictionary_file)
        
    
    def getRelevance(self, document_id, term):
        try:
            return self.docs[document_id].tf_text[term] * self.idf[term]
        except: 
            return 0

<h2>Create Document Collection and Query Collection objects</h2>

In [8]:
recompute_all = True
#seed for random number
seed = 1
doc_list = getDocumentList(collection_path)

#Compute or read document collection
compute_document_collection = False or recompute_all

#create a Document collection object
document_collection = DocumentCollection(collection_path, doc_list)
if compute_document_collection:
    start = time.time()
    document_collection.computeAndDump(doc_list)
    end = time.time()
    print('Document Collection Computed in ' + str(round(end - start, 4)) + 's')
else:
    start = time.time()
    document_collection.loadFromFile()
    end = time.time()
    print('Document Collection Loaded in ' + str(round(end - start, 4)) + 's')

Document Collection Computed in 4.8241s


<h2>Classes for ranking</h2>

In [9]:
class RankResult:
    def __init__(self, query, img_dir, relevance):
        self.query = query
        self.img_dir = img_dir
        self.relevance = relevance
        
class RankingModel:
    def __init__(self, document_collection):
        self.document_collection = document_collection
    
    def getQueryResult(self, query, limit_result=20):
        rank = []
        search_space = set()
        for term in processText(query):
            if term in self.document_collection.inverted_index:
                search_space = search_space.union(set(self.document_collection.inverted_index[term]))
        for doc_id in search_space:
            rank += [RankResult(query, self.document_collection.docs[doc_id].img_dir, self.calculateRelevance(self.document_collection.docs[doc_id], query))] 
        rank.sort(key=lambda x: x.relevance, reverse=True)
        return rank[0:limit_result]
    
    def calculateRelevance(self, document, query):
        return 0

<h2>Vector space model VSM</h2>

In [10]:
class VectorSpaceModel(RankingModel):
    def __init__(self, document_collection):
        super().__init__(document_collection)
        
    def vectorizeXmlQuery(self, query):
        return [self.getQueryRelevance(query, t) for t in query]
    
    def vectorizeDocument(self, document, query):
        return [self.getDocumentRelevance(document, t) for t in query]

    #tf-idf
    def getDocumentRelevance(self, document, term):
        return self.document_collection.getRelevance(document.id, term)
    
    def getQueryRelevance(self, query, term):
        return query.count(term)/len(query)

    def dotSimilarity(self, vect1, vect2):
        return np.dot(vect1, vect2)
    
    def calculateRelevance(self, document, query, similarity_function=lambda v1, v2: np.dot(v1, v2)):
        v_query = self.vectorizeXmlQuery(query)
        v_doc = self.vectorizeDocument(document, query)        
        return similarity_function(v_doc, v_query)

<h2>Generate the VSM report for trec_eval</h2>

In [11]:
vsm = VectorSpaceModel(document_collection)
query = 'alligator'
vsm.getQueryResult(query)[0].img_dir
#document_collection.docs['1'].img_dir

'./simple_images/Alligators/Alligators_4.jpeg'

<h2>BM25 ranking</h2>
<p>For a query $Q$ and a term $t \in Q$, the <b>BM25</b> score for a document $D$ is: </p>
<h1>$Relevance(D,t) = IDF(t) \cdot \frac{count(D, t) \cdot (k + 1)}{count(D, t) + k \cdot (1 - b + \frac{b|D|}{avgdl})} $</h1>
<p>Where $count(D, t)$ is the number of occurrences of $t$ in $D$</p>

In [12]:
class BM25(VectorSpaceModel):
    def __init__(self, document_collection, k = 1.2, b = .75):
        super().__init__(document_collection)
        self.k = k
        self.b = b
    
    def getDocumentRelevance(self, document, term):
        try:
            idf_term = self.document_collection.idf[term]
            #x = number of occurencies of term in document
            x = document.tf_text[term] * len(document.processed_body+document.processed_head)
        except:
            return 0
        doc_len = len(document.processed_head + document.processed_body)
        normalizer = 1 - self.b + (self.b * doc_len / self.document_collection.avgdl)
        return (self.k + 1) * x / (x + self.k * normalizer) * idf_term

<h2>Generate the BM25 report for trec_eval</h2>

In [13]:
bm25 = BM25(document_collection)
vsm.getQueryResult(query)[0].img_dir

'./simple_images/Alligators/Alligators_4.jpeg'

<h2>BM25F</h2>
<p>In order to attribute different relevance with respect to the document fields, it is possible to use a weighted version of the $tf(D, t)$ function such that:</p>
<h2>$tf(D, t) = \sum_{c \in D} w_c \cdot tf_c(D, t)$</h2>
<p>where:</p>
<ul>
    <li>$c$ is a document field</li>
    <li>$w_c$ is the weight attributed to field c</li>
    <li>$tf_c(D, t)$ is the term frequency for the field $c$ </li>
</ul>

In [14]:
class BM25F(VectorSpaceModel):
    def __init__(self, document_collection, k=1.2, b=.75,
                w_head=3, w_body=1):
        super().__init__(document_collection)
        self.w_head = w_head
        self.w_body = w_body
        self.k = k
        self.b = b
    
    def getDocumentRelevance(self, document, term):
        try:
            idf_term = self.document_collection.idf[term]
        except:
            return 0
        x_head = document.tf_head[term] * len(document.processed_head) * self.w_head if term in document.tf_head else 0
        x_body = document.tf_body[term] * len(document.processed_body) * self.w_body if term in document.tf_body else 0
        x = x_head + x_body
        
        doc_len = len(document.processed_head + document.processed_body)
        normalizer = 1 - self.b + (self.b * doc_len / self.document_collection.avgdl)
        return (self.k + 1) * x / (x + self.k * normalizer) * idf_term
    

<h2>Generate the BM25F report for trec_eval</h2>

In [15]:
bm25f = BM25F(document_collection, w_head=3, w_body=1)
bm25f.getQueryResult(query)[0].img_dir

'./simple_images/Alligators/Alligators_4.jpeg'

<h2>Unigram Language Model</h2>
<p>A unigram language model does not consider the context and estimates each term independently. 
    As a result:
    $P_{uni}(t_1 t_2 t_3 t_4) = P(t_1)P(t_2)P(t_3)P(t_4)$
</p>

<p>It is possible to consider a document $d$ as a generative model $M_d$ s.t. $\sum_{t}P(t|M_d) = 1$</p>
<p>Given a query $q$ we rank documents exploiting the likelihood of the document model to generate $q: P(q|M_d)$.</p>
<p><b>Maximum likelihood estimate (MLE)</b> for a query $q = [t_1, \dots, t_n]$ and a generative model $M_d$, $P(t_1, \dots, t_n | M_d) = tf(d, t_1) \times \dots \times tf(d, t_n)$</p>
<p><b>Zero Probability Problem: </b>if a term $t_h \in q$ is s.t. $tf(d, t_h) = 0$ hence $P(q|M_d) = 0$</p>
<p>To overcome this problem, only query term that are present in the document will be attributed a probability, the probability of the total seen terms is normalized to $1$</p>
<p><b>Over Estimation Problem: </b> since with MLE only terms belonging to $q \cap d$ are estimated, if there is only one common term between document and query, i.e. $|q \cap d| = 1$, the relevance would be $1$</p>
<p>To overcome this second problem, it is common to attribute a mass weight to other terms in the document i.e. <b>smoothing</b>.</p>
<p><b>Linear smoothing: </b> given a document model $M_d$ and a collection model $M_c$:</p>
<h2>$P(t|M_d) = \lambda \frac{tf(d, t)}{|d|} + (1 - \lambda) P(t|M_c)$</h2>
<p>where $\lambda$ is a parameter s.t. $\lambda \in (0, 1)$ and $P(t|M_c)$ is the term frequency of $t$ in the entire collection of documents</p>
<p>Note: for high values of $\lambda$ the search is more <i>conjunctive</i> i.e. favour documents containing all query terms, for low values of $\lambda$ the search is more <i>disjunctive</i> i.e. more suitable for long queries. Tuning this parameter is collection-specific.</p>
<p><b>Dirichlet Smoothing: </b>more effective in IR, sets <font size="+2">$\lambda = \frac{|d|}{\alpha + |d|}$</font> where $\alpha$ is the background mass i.e. the number of terms not in $q \cap d$</p>
<p>Finally: </p>
<h2>$P(q|d) = \prod_{t \in q} (\lambda \frac{tf(d,t)}{|d|} + (1-\lambda) \frac{tf(c, t)}{|c|}) = $</h2>
<h2>$\prod_{t \in q} (\frac{|d|}{\alpha + |d|} \frac{tf(d,t)}{|d|} + \frac{\alpha}{\alpha + |d|} \frac{tf(c, t)}{|c|}) = \prod_{t \in q} ( \frac{tf(d,t)}{\alpha + |d|} + \frac{\alpha}{\alpha + |d|} \frac{tf(c, t)}{|c|})$</h2>
<p>Using logs to avoid underflow in computation since $log(xy) = log(x) + log(y)$: </p>
<h2>$log(P(q|d)) = \sum_{t \in q} log( \frac{tf(d,t)}{\alpha + |d|} + \frac{\alpha}{\alpha + |d|} \frac{tf(c, t)}{|c|})$</h2>
<p><b>Problem: </b>if a term $t$ is not present in the entire document collection, ranking of documents would be $- \infty$, if <font size="+1">$(\frac{tf(d,t)}{\alpha + |d|} + \frac{\alpha}{\alpha + |d|} \frac{tf(c, t)}{|c|}) = 0$</font> for a term $t$ that term will not be considered</p>

In [16]:
class UnigramLanguageModel(RankingModel):
    def __init__(self, document_collection):
        super().__init__(document_collection)
    
    def calculateRelevance(self, document, query):
        query = processText(query)
        relevance = 0
        d_len = len(document.processed_head + document.processed_body)
        alpha = len([t for t in document.processed_head + document.processed_body if t not in query])
        for t in query:
            #must multiply * d_len because of the tf_text formulation in XmlDocument class
            first_term = document.tf_text[t] * d_len if t in document.tf_text else 0
            second_term = alpha * self.document_collection.global_tf[t] if t in self.document_collection.global_tf else 0#global_tf includes |c|
            denumerator = (d_len + alpha)
            result = (first_term + second_term) / denumerator
            relevance += np.log(result) if result != 0 else 0 #if a term in a query is not present in entire document collection, relevance is -inf
        return relevance

<h2>Generate the ULM report for trec_eval</h2>

In [17]:
ulm = UnigramLanguageModel(document_collection)
ulm.getQueryResult(query)[0].img_dir

'./simple_images/Alligators/Alligators_6.jpeg'

# Python Server

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def home():
    model = request.args.get('model') if request.args.get('model') is not None else 'VSM'
    q = request.args.get('q') if request.args.get('q') is not None else ''
    limit = int(request.args.get('limit')) if request.args.get('limit') is not None else 10

    vsm_c, bm25_c, ulm_c = '', '', ''
    
    if model == 'VSM':
        vsm_c = 'checked'
        q_res = [r.img_dir[1:] for r in vsm.getQueryResult(q, limit)]
    
    if model == 'BM25':
        bm25_c = 'checked'
        q_res = [r.img_dir[1:] for r in bm25.getQueryResult(q, limit)]
    
    if model == 'ULM':
        ulm_c = 'checked'
        q_res = [r.img_dir[1:] for r in ulm.getQueryResult(q, limit)]
    
    return f"""
    
        <!DOCTYPE html>
        <html>
        <head>
            <title>Image Retrieval WebApp</title>
            <link href="static/style.css" rel="stylesheet" />
            <link rel="preconnect" href="https://fonts.googleapis.com">
            <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
            <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap" rel="stylesheet"> 
        </head>
        <body>
            <h2>Image Search Engine</h2>
            <form id="search-commands">
            <input placeholder="Type your search terms" type="text" name="q" value="{q}" />
            <input type="submit" value="search images" /> <br><br>
            <b>Select engine</b>:
            <span class="eng">
                <input type="radio" name="model"  value="VSM" {vsm_c}> 
                <label for="VSM">VSM <i>tf-idf</i></label>
            </span>
            <span class="eng">
                <input type="radio" name="model" value="BM25" {bm25_c}>
                <label for="BM25">BM25</label>
            </span>
            <span class="eng">
                <input type="radio" name="model"  value="ULM" {ulm_c}> 
                <label for="ULM">ULM</label>
            </span>
            </form>
            <i>Don't know what to search?</i> <span id="myBtn">List of topics</span>
            <div id="img-container">
            {rend_imgs(q_res)}
            </div>
            <div id="myModal" class="modal">

              <!-- Modal content -->
              <div class="modal-content">
                <span class="close">&times;</span>
                <p>{topics_text()}</p>
              </div>

            </div>
        </body>
        <script>
        var modal = document.getElementById("myModal");
        var btn = document.getElementById("myBtn");
        var span = document.getElementsByClassName("close")[0];
        btn.onclick = function() {{
          modal.style.display = "block";
        }}

        // When the user clicks on <span> (x), close the modal
        span.onclick = function() {{
          modal.style.display = "none";
        }}

        // When the user clicks anywhere outside of the modal, close it
        window.onclick = function(event) {{
          if (event.target == modal) {{
            modal.style.display = "none";
          }}
        }}
        </script>
        </html>

    """

def topics_text():
    #Read topics from file (100 topics)
    topics = open("topics.txt", "r")
    content_list = topics.readlines()
    topics.close()
    #Prepare topics for downloader
    content_list = ['<span class="topic-entry">' + el.strip() + '</span>' for el in content_list]
    topics_search_str = ' '.join(content_list)
    return topics_search_str

def rend_imgs(img_list):
    if len(img_list) == 0:
        return 'No results found..'
    r = ''
    for el in img_list:
        r += '<img height="150" src="static' + el + '" alt="User Image">'
    return r

if __name__ == '__main__':
    # Run the app server on localhost:4449
    app.run('localhost', 4449)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:4449/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /?q=teachiing+and+learning&model=ULM HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching%20and%20learning_6.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching%20and%20learning_9.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching%20and%20learning_4.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching%20and%20learning_5.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching%20and%20learning_7.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:49:49] "GET /static/simple_images/Teaching_and_learning/Teaching

127.0.0.1 - - [11/Mar/2022 15:52:52] "GET /static/simple_images/Children_and_family/Children%20and%20family_9.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:52:52] "GET /static/simple_images/Children_and_family/Children%20and%20family_2.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:52:52] "GET /static/simple_images/Children_and_family/Children%20and%20family_5.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:52:52] "GET /static/simple_images/Children_and_family/Children%20and%20family_10.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:52:52] "GET /static/simple_images/Children_and_family/Children%20and%20family_1.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:52:56] "GET /?q=family&model=BM25 HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 15:52:56] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:52:56] "GET /static/simple_images/Children_and_family/Children%20and%20family_6.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 15:52:56] "GET /static/simple_